In [237]:
import os

from flask import Flask, render_template, request, jsonify
#from flask_cors import CORS, cross_origin
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq

In [238]:
searchString = 'apple'   ## Enter the mobile brand name
flipkart_url = "https://www.flipkart.com/search?q=" + searchString  ## Create url to search 
uClient = uReq(flipkart_url)  ## Open the search url
flipkartPage = uClient.read() ## Read the data
uClient.close()  ## Close the url

flipkart_html = bs(flipkartPage, "html.parser")   ## convert the html data to undestandable format by the BeautifulSoup
models = flipkart_html.findAll("div", {"class": "bhgxx2 col-12-12"})  ## Finds all the mobile models in the parsed html data
# models = flipkart_html.find_all("div", {"class": "bhgxx2 col-12-12"})
list_reviews = []  ## Create an empty list to store the review comment details

for i in range(7):  ## Select first 7 mobile models to loop through. 7 can be repladed with models to loop through all the models
    try:
        
        modellink = 'https://www.flipkart.com'+ models[i].div.div.div.a['href']
        modelopen = requests.get(modellink)
        modelopen.encoding = 'utf-8'
        model_html = bs(modelopen.text, "html.parser")
        model_review_link = model_html.find('div', {'class': 'swINJg _3nrCtb'})
        model_name = model_html.find_all('span', {'class': '_35KyD6'})[0].text.split('(')[0]
        
        if model_review_link != None:
            all_review_link = 'https://www.flipkart.com'+ model_review_link.find_parent().get('href')
            #print(all_review_link)
            review_open = requests.get(all_review_link)
            review_open_html = bs(review_open.content, "html.parser")
            num_pages = review_open_html.find('div', {'class': '_2zg3yZ _3KSYCY'})
            if num_pages != None:
                s = num_pages.find('span').get_text().split()[-1]
                pages = int(''.join(i for i in s if i.isdigit()))

                for i in range(1,2):   # variable 'pages' has the actual number of pages. 
                    url = all_review_link + '&page=' + '{}'.format(i)
                    req_data = requests.get(url)
                    review_soup = bs(req_data.content, "html.parser")
                    #all_reviews = review_soup.find_all('div', {'class': '_3gijNv col-12-12'})
                    all_reviews = review_soup.find_all('div', {'class': '_1PBCrt'})
                    count = 1

                    for comment in all_reviews:
                        count +=1
                        try:
                            name = comment.find_all('p', {'class': '_3LYOAd _3sxSiS'})[0].text
                            #name = comment.find_all('p', {'class': '_3LYOAd _3sxSiS'})[0].text
                        except:
                            name = 'No name'

                        try:
                            rating = comment.div.div.div.div.text
                        except:
                            reating = 'No Rating'

                        try:
                            commenthead = comment.div.div.div.p.text
                        except:
                            commenthead = 'No Comment Heading'

                        try:
                            #custComment = comment.find_all('div', {'class': 'qwjRop'})[0].text
                            custComment = comment.div.div.find_all('div', {'class': ''})[1].text
                        except Exception as e:
                            print("Exception while creating dictionary: ", e)

                        mydict = {"Product": model_name, "Name": name, "Rating": rating, "CommentHead": commenthead,
                                              "Comment": custComment}      

                        list_reviews.append(mydict)


            else:
                reviews = model_html.find_all('div', {'class': '_3nrCtb'}) 
                for comment in reviews:
                    try:
                        name = comment.div.div.find_all('p', {'class': '_3LYOAd _3sxSiS'})[0].text
                        #name = comment.find_all('p', {'class': '_3LYOAd _3sxSiS'})[0].text
                    except:
                        name = 'No name'

                    try:
                        rating = comment.div.div.div.div.text
                    except:
                        reating = 'No Rating'

                    try:
                        commenthead = comment.div.div.div.p.text
                    except:
                        commenthead = 'No Comment Heading'

                    try:
                        comtag = comment.div.div.find_all('div', {'class': ''})
                        custComment = comtag[1].text
                    except Exception as e:
                        print("Exception while creating dictionary: ", e)


                    mydict = {"Product": model_name, "Name": name, "Rating": rating, "CommentHead": commenthead,
                                          "Comment": custComment}


                    list_reviews.append(mydict)
                    #modellink

        if model_review_link == None:
                reviews = model_html.find_all('div', {'class': '_3nrCtb'}) 
                for comment in reviews:
                    try:
                        name = comment.div.div.find_all('p', {'class': '_3LYOAd _3sxSiS'})[0].text
                        #name = comment.find_all('p', {'class': '_3LYOAd _3sxSiS'})[0].text
                    except:
                        name = 'No name'

                    try:
                        rating = comment.div.div.div.div.text
                    except:
                        reating = 'No Rating'

                    try:
                        commenthead = comment.div.div.div.p.text
                    except:
                        commenthead = 'No Comment Heading'

                    try:
                        comtag = comment.div.div.find_all('div', {'class': ''})
                        custComment = comtag[1].text
                    except Exception as e:
                        print("Exception while creating dictionary: ", e)


                    mydict = {"Product": model_name, "Name": name, "Rating": rating, "CommentHead": commenthead,
                                          "Comment": custComment}


                    list_reviews.append(mydict)
                    #modellink  

    except:
        #print('Exception message as: ', e)
        continue                

In [239]:
list_reviews

[{'Product': 'Apple iPhone SE ',
  'Name': 'ankit paliwal',
  'Rating': '5',
  'CommentHead': 'Best in the market!',
  'Comment': "One word.. beautiful iphone ❤️ premium feelJust go for it without any doubt. Superb camera, handy, white color is soo good. This phone is for someone who don't like big screen phones."},
 {'Product': 'Apple iPhone SE ',
  'Name': 'Prashant Kumar Sharma',
  'Rating': '1',
  'CommentHead': 'Unsatisfactory',
  'Comment': 'Everything is fine of this mobile except battery. Battery consumption at 8-10% Brightness inVideo call : almost 4% reduction for every 5-6 mins. Phone call : almost 1% reduction for every 4-5 mins. Which seems to be bit faster consumption. First iPhone I’m using. I’m not sure about regular battery consumption rate. But this doesn’t look nice to me.'},
 {'Product': 'Apple iPhone SE ',
  'Name': 'Rawat Ns',
  'Rating': '5',
  'CommentHead': 'Wonderful',
  'Comment': 'Like It, Apple SE very nice and handy mobile, we can use it by single hand, li